<a href="https://colab.research.google.com/github/suchanya-pangam/670510749-229352-StatisticalLearning-or-Statistical-Learning-Labs./blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [2]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [3]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [4]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [5]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes': 1,'no': 0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop(['y', 'y_new'], axis=1)

print(x)
print(y)



       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      no  yes  telephone   may         mon       307         1    999         0  nonexistent
...    ...          ...      ...                  ...     ...     ...  ...  

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
20031,59,retired,married,basic.4y,no,yes,no,cellular,aug,mon,67,6,999,0,nonexistent
4615,58,retired,married,university.degree,no,no,no,telephone,may,tue,98,2,999,0,nonexistent
4385,32,services,married,high.school,no,no,no,telephone,may,tue,140,1,999,0,nonexistent
5082,47,blue-collar,married,basic.4y,NaN,no,no,telephone,may,fri,107,1,999,0,nonexistent
39712,41,management,married,NaN,no,yes,no,cellular,may,fri,588,1,18,2,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22359,32,services,married,professional.course,NaN,yes,no,cellular,aug,thu,160,7,999,0,nonexistent
39757,35,unemployed,married,high.school,no,yes,no,telephone,may,fri,43,1,999,2,failure
24111,33,self-employed,single,university.degree,no,no,no,telephone,nov,thu,168,1,999,0,nonexistent
4609,42,admin.,married,high.school,no,no,no,telephone,may,tue,1392,2,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [8]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo])
x_test[num_cols_demo] = scaler.transform(x_test[num_cols_demo])
# Fit the scaler ONLY on the training data



In [9]:
x_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,1.651221e-17,258.879678,-2.834186e-17,962.592869,0.172280
std,1.000017e+00,260.913546,1.000017e+00,186.628954,0.497419
min,-2.202556e+00,0.000000,-5.699568e-01,0.000000,0.000000
25%,-7.656447e-01,102.000000,-5.699568e-01,999.000000,0.000000
50%,-1.908804e-01,180.000000,-2.072021e-01,999.000000,0.000000
75%,6.712661e-01,319.000000,1.555526e-01,999.000000,0.000000
max,5.556763e+00,4918.000000,1.938155e+01,999.000000,7.000000


In [10]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(x_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(pd.DataFrame(x_train[num_cols_demo], columns=num_cols_demo, index=x_train.index).head())

print("\nMean of scaled 'age' (train):", x_train[num_cols_demo].iloc[:, 0].mean())
print("Std Dev of scaled 'campaign' (train):", x_train[num_cols_demo].iloc[:, 1].std())


Original X_train 'age' and 'campaign' head:
            age  campaign
22470 -0.574057  1.606571
19864 -1.053027 -0.569957
9335   0.383884 -0.207202
17052  0.192296 -0.207202
34133 -0.957233  0.155553

Scaled X_train 'age' and 'campaign' head:
            age  campaign
22470 -0.574057  1.606571
19864 -1.053027 -0.569957
9335   0.383884 -0.207202
17052  0.192296 -0.207202
34133 -0.957233  0.155553

Mean of scaled 'age' (train): 1.6512213692180885e-17
Std Dev of scaled 'campaign' (train): 1.0000173428949612


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [11]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here
x_train[ord_col_demo] = imputer_ord.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo] = imputer_ord.transform(x_test[ord_col_demo])
x_train['education']



,education
22470,university.degree
19864,professional.course
9335,university.degree
17052,professional.course
34133,high.school
...,...
33125,university.degree
32051,basic.4y
27699,basic.6y
18276,basic.6y


- **Ordinal Encoding**

In [12]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [13]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here
x_train[ord_col_demo] = ordinal_encoder.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo] = ordinal_encoder.transform(x_test[ord_col_demo])

x_test





,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
20031,1.820795,retired,married,1.0,no,yes,no,cellular,aug,mon,67,1.243817,999,0,nonexistent
4615,1.725001,retired,married,6.0,no,no,no,telephone,may,tue,98,-0.207202,999,0,nonexistent
4385,-0.765645,services,married,4.0,no,no,no,telephone,may,tue,140,-0.569957,999,0,nonexistent
5082,0.671266,blue-collar,married,1.0,NaN,no,no,telephone,may,fri,107,-0.569957,999,0,nonexistent
39712,0.096502,management,married,6.0,no,yes,no,cellular,may,fri,588,-0.569957,18,2,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22359,-0.765645,services,married,5.0,NaN,yes,no,cellular,aug,thu,160,1.606571,999,0,nonexistent
39757,-0.478263,unemployed,married,4.0,no,yes,no,telephone,may,fri,43,-0.569957,999,2,failure
24111,-0.669851,self-employed,single,6.0,no,no,no,telephone,nov,thu,168,-0.569957,999,0,nonexistent
4609,0.192296,admin.,married,4.0,no,no,no,telephone,may,tue,1392,-0.207202,999,0,nonexistent


In [14]:
print("\nOriginal X_train 'education' head:")
print(x_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x_train.index).iloc[20:25])


Original X_train 'education' head:
       education
15638        6.0
40918        5.0
8578         5.0
31354        6.0
25649        4.0

Imputed X_train 'education' head (after imputer.transform):
       education
15638        6.0
40918        5.0
8578         5.0
31354        6.0
25649        4.0

Ordinal Encoded X_train 'education' head:
       education
15638        6.0
40918        5.0
8578         5.0
31354        6.0
25649        4.0


**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [16]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train[nom_col_demo])

X_train_imputed_nom_demo = imputer_nom.transform(x_train[nom_col_demo])
X_test_imputed_nom_demo = imputer_nom.transform(x_test[nom_col_demo])

- **Nominal Encoding**

In [17]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

x_train_onehot = onehot_encoder.fit_transform(x_train[['job']])
x_test_onehot = onehot_encoder.transform(x_test[['job']])

x_train_onehot.shape




(28831, 12)

In [18]:
x_train_onehot

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [19]:
x_train = pd.concat([x_train.reset_index(drop=True),
                     pd.DataFrame(x_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
x_train = x_train.drop(['job'], axis=1)

x_train

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
0,-0.574057,married,6.0,no,no,yes,cellular,aug,fri,944,1.606571,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.053027,married,5.0,no,yes,no,cellular,aug,fri,112,-0.569957,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.383884,married,6.0,NaN,yes,yes,telephone,jun,fri,545,-0.207202,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.192296,married,5.0,no,yes,no,cellular,jul,fri,154,-0.207202,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.957233,single,4.0,no,yes,no,cellular,may,wed,202,0.155553,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,0.288090,married,6.0,no,yes,no,cellular,may,tue,756,0.155553,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,-0.095086,married,1.0,no,no,no,cellular,may,thu,155,-0.569957,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,0.767060,married,2.0,no,no,no,telephone,mar,mon,139,0.881062,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28829,0.288090,married,2.0,NaN,yes,no,cellular,jul,wed,61,0.155553,999,0,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
x_test = pd.concat([x_test.reset_index(drop=True),
                    pd.DataFrame(x_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
0,1.820795,retired,married,1.0,no,yes,no,cellular,aug,mon,67,1.243817,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.725001,retired,married,6.0,no,no,no,telephone,may,tue,98,-0.207202,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.765645,services,married,4.0,no,no,no,telephone,may,tue,140,-0.569957,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.671266,blue-collar,married,1.0,NaN,no,no,telephone,may,fri,107,-0.569957,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.096502,management,married,6.0,no,yes,no,cellular,may,fri,588,-0.569957,18,2,failure,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,-0.765645,services,married,5.0,NaN,yes,no,cellular,aug,thu,160,1.606571,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
12353,-0.478263,unemployed,married,4.0,no,yes,no,telephone,may,fri,43,-0.569957,999,2,failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12354,-0.669851,self-employed,single,6.0,no,no,no,telephone,nov,thu,168,-0.569957,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12355,0.192296,admin.,married,4.0,no,no,no,telephone,may,tue,1392,-0.207202,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
print("\nOriginal X_train 'job' head:")
# The 'job' column was dropped from x_train in a previous step, so trying to access it directly will cause a KeyError.
# If you want to see the original imputed job values, use X_train_imputed_nom_demo.
# print(x_train[nom_col_demo].iloc[40:45]) # This line causes the KeyError
print("\nImputed X_train 'job' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=x_train.index).iloc[40:45])
print("\nOne-Hot Encoded X_train 'job' shape:", x_train_onehot.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
print(pd.DataFrame(x_train_onehot, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=x_train.index).iloc[40:45])


Original X_train 'job' head:

Imputed X_train 'job' head (after imputer.transform):
             job
40       retired
41    management
42  entrepreneur
43   blue-collar
44        admin.

One-Hot Encoded X_train 'job' shape: (28831, 12)
First 5 rows of One-Hot Encoded X_train 'job':
    job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  job_management  job_retired  job_self-employed  job_services  job_student  job_technician  job_unemployed  job_nan
40         0.0              0.0               0.0            0.0             0.0          1.0                0.0           0.0          0.0             0.0             0.0      0.0
41         0.0              0.0               0.0            0.0             1.0          0.0                0.0           0.0          0.0             0.0             0.0      0.0
42         0.0              0.0               1.0            0.0             0.0          0.0                0.0           0.0          0.0             0.0             0.0  

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [22]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()


Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


In [23]:
numerical_features = ['age', 'duration', 'campaign', 'pdays', 'previous']
ordinal_features = ['education']
nominal_features = ['marital', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'day_of_week'] # Updated to include all nominal features

In [24]:

x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

In [25]:
scaler = StandardScaler()

x_train_copy[numerical_features] = scaler.fit_transform(x_train_copy[numerical_features]) # Apply to all numerical_features
x_test_copy[numerical_features] = scaler.transform(x_test_copy[numerical_features]) # Apply to all numerical_features

In [26]:
x_train_copy.describe()


,age,education,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
count,2.883100e+04,28831.000000,2.883100e+04,2.883100e+04,2.883100e+04,2.883100e+04,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000
mean,1.281545e-17,4.246471,5.027599e-17,-1.601931e-17,-1.804021e-16,1.133674e-17,0.253963,0.224446,0.034685,0.026048,0.070584,0.041865,0.034650,0.096251,0.021262,0.162915,0.025389,0.007943
std,1.000017e+00,1.640396,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,0.435284,0.417224,0.182984,0.159282,0.256133,0.200283,0.182895,0.294940,0.144258,0.369294,0.157307,0.088769
min,-2.202556e+00,0.000000,-9.922220e-01,-5.699568e-01,-5.157879e+00,-3.463534e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.656447e-01,3.000000,-6.012812e-01,-5.699568e-01,1.950810e-01,-3.463534e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-1.908804e-01,4.000000,-3.023264e-01,-2.072021e-01,1.950810e-01,-3.463534e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.712661e-01,6.000000,2.304264e-01,1.555526e-01,1.950810e-01,-3.463534e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.556763e+00,6.000000,1.785726e+01,1.938155e+01,1.950810e-01,1.372653e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
x_train_copy['education'].unique()

array([6., 5., 4., 3., 1., 2., 0.])

In [28]:
x_train_copy['education'] = x_train_copy['education'].astype(float)
x_test_copy['education'] = x_test_copy['education'].astype(float)

In [29]:
x_train_copy

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
0,-0.574057,married,6.0,no,no,yes,cellular,aug,fri,2.625897,1.606571,0.195081,-0.346353,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.053027,married,5.0,no,yes,no,cellular,aug,fri,-0.562954,-0.569957,0.195081,-0.346353,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.383884,married,6.0,NaN,yes,yes,telephone,jun,fri,1.096629,-0.207202,0.195081,-0.346353,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.192296,married,5.0,no,yes,no,cellular,jul,fri,-0.401978,-0.207202,0.195081,-0.346353,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.957233,single,4.0,no,yes,no,cellular,may,wed,-0.218006,0.155553,0.195081,-0.346353,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,0.288090,married,6.0,no,yes,no,cellular,may,tue,1.905340,0.155553,0.195081,-0.346353,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,-0.095086,married,1.0,no,no,no,cellular,may,thu,-0.398145,-0.569957,0.195081,-0.346353,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,0.767060,married,2.0,no,no,no,telephone,mar,mon,-0.459469,0.881062,0.195081,-0.346353,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28829,0.288090,married,2.0,NaN,yes,no,cellular,jul,wed,-0.758424,0.155553,0.195081,-0.346353,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
x_train_copy.columns.tolist()

['age',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'job_admin.',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_nan']

In [31]:
x_test_copy.columns.tolist()

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'job_admin.',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_nan']

In [32]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


x_train_copy, x_test_copy, y_train_exercise, y_test_exercise = train_test_split(x.copy(), y.copy(), test_size=0.3, random_state=42)

# --- Drop 'duration' from here, as the original 'x' still contains it ---
x_train_copy = x_train_copy.drop('duration', axis=1)
x_test_copy = x_test_copy.drop('duration', axis=1)

# Define feature lists for this exercise (updated with 'duration' removed from numerical)
numerical_features = ['age', 'campaign', 'pdays', 'previous']
ordinal_features = ['education']
nominal_features = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

# --- 1. Numerical Feature Preprocessing (Standard Scaling) ---
scaler_exercise = StandardScaler()
x_train_copy[numerical_features] = scaler_exercise.fit_transform(x_train_copy[numerical_features])
x_test_copy[numerical_features] = scaler_exercise.transform(x_test_copy[numerical_features])

# --- 2. Ordinal Feature Preprocessing (Imputation & Encoding) ---

# Impute missing values for ordinal features
imputer_ord_exercise = SimpleImputer(strategy='most_frequent')
x_train_copy[ordinal_features] = imputer_ord_exercise.fit_transform(x_train_copy[ordinal_features])
x_test_copy[ordinal_features] = imputer_ord_exercise.transform(x_test_copy[ordinal_features])

# Define education categories (re-defining for robustness)
education_categories_correct = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

ord_enc_exercise = OrdinalEncoder(categories=[education_categories_correct], handle_unknown='use_encoded_value', unknown_value=-1)
x_train_copy[ordinal_features] = ord_enc_exercise.fit_transform(x_train_copy[ordinal_features])
x_test_copy[ordinal_features] = ord_enc_exercise.transform(x_test_copy[ordinal_features])

# --- 3. Nominal Feature Preprocessing (Imputation & One-Hot Encoding) ---

# Imputation for nominal features
imputer_nom_exercise = SimpleImputer(strategy='most_frequent')
x_train_copy[nominal_features] = imputer_nom_exercise.fit_transform(x_train_copy[nominal_features])
x_test_copy[nominal_features] = imputer_nom_exercise.transform(x_test_copy[nominal_features])

# One-Hot Encoding for nominal features
ohe_exercise = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

x_train_ohe = pd.DataFrame(ohe_exercise.fit_transform(x_train_copy[nominal_features]), index=x_train_copy.index)
x_test_ohe = pd.DataFrame(ohe_exercise.transform(x_test_copy[nominal_features]), index=x_test_copy.index)

x_train_ohe.columns = ohe_exercise.get_feature_names_out(nominal_features)
x_test_ohe.columns = ohe_exercise.get_feature_names_out(nominal_features)

# Concat one-hot encoded features and drop original nominal columns
x_train_copy = pd.concat([x_train_copy.drop(columns=nominal_features), x_train_ohe], axis=1)
x_test_copy = pd.concat([x_test_copy.drop(columns=nominal_features), x_test_ohe], axis=1)


In [33]:
x_train_copy

,age,education,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
39075,-1.053709,6.0,0.154139,0.193350,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34855,-1.053709,6.0,0.517478,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7107,0.477399,2.0,-0.209199,0.193350,-0.349144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
31614,-0.575238,6.0,-0.572537,0.193350,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34878,-0.766626,4.0,2.334170,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1.721425,5.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
11284,-0.288155,6.0,-0.572537,0.193350,-0.349144,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
38158,-0.479543,4.0,-0.572537,-5.182867,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
860,-0.001072,6.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [34]:
x_test_copy

,age,education,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
32884,1.625730,4.0,-0.572537,0.193350,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3169,1.434342,6.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
32206,-0.670932,3.0,-0.572537,0.193350,1.673800,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9403,-0.383849,4.0,0.517478,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
14020,-1.245098,4.0,-0.209199,0.193350,-0.349144,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15908,0.573094,4.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
28222,-0.192461,4.0,-0.572537,-5.193674,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
14194,-1.340792,4.0,-0.209199,0.193350,-0.349144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
19764,1.051565,5.0,0.154139,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 1) Train model
lr = LogisticRegression(max_iter=1000, solver='lbfgs')
lr.fit(x_train_copy, y_train)

# 2) Predict on test set
y_pred = lr.predict(x_test_copy)

# 3) Evaluation
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     11012
           1       0.00      0.00      0.00      1345

    accuracy                           0.89     12357
   macro avg       0.45      0.50      0.47     12357
weighted avg       0.79      0.89      0.84     12357



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [36]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Fashion-MNIST Train data shape: (60000, 28, 28)
Fashion-MNIST Train labels shape: (60000,)
Fashion-MNIST Test data shape: (10000, 28, 28)
Fashion-MNIST Test labels shape: (10000,)


In [37]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

First image [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


